SECTION 1: IMPORTS AND SETUP


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score

from math import radians, cos, sin, asin, sqrt
import folium
from folium import plugins

SECTION 2: DATA LOADING FUNCTIONS


In [ ]:
print("="*80)
print("STEP 1: LOADING DATA")
print("="*80 + "\n")

# Load traffic data
traffic_df = pd.read_csv('/content/drive/MyDrive/DA_Project/cleaned/traffic_feb2024_sample.csv')
stops_df = pd.read_csv('/content/drive/MyDrive/DA_Project/cleaned/cleaned_bus_stops_file.csv')
routes_df = pd.read_csv('/content/drive/MyDrive/DA_Project/cleaned/cleaned_bus_routes_file.csv')

print(f"Loaded {len(traffic_df):,} traffic records")
print(f"Loaded {len(stops_df)} bus stops")
print(f"Loaded {len(routes_df)} bus routes\n")


STEP 1: LOADING DATA

Loaded 1,549,600 traffic records
Loaded 4674 bus stops
Loaded 578 bus routes



In [ ]:
routes_df

,route_id,name,ref,route_type,num_stops,segment_distance_list,total_distance_km,coordinates
0,1700755,เมืองทอง-แจ้งวัฒนะ,1,bus,4,"[0.252, 0.99, 0.019]",1.260725,"[[100.5357615, 13.9020239], [100.5380964, 13.9..."
1,1701250,แจ้งวัฒนะ-ติวานนท์,3,bus,4,"[1.001, 0.019, 1.04]",2.060307,"[[100.5380964, 13.9020271], [100.5386583, 13.9..."
2,1701251,166 เมืองทองธานี - อนุสาวรีย์ชัยสมรภูมิ,166,bus,25,"[0.36, 13.825, 1.214, 16.052, 1.899, 14.35, 1....",118.144379,"[[100.5357615, 13.9020239], [100.5388988, 13.9..."
3,2148296,รถโดยสารประจำทางด่วนพิเศษ 402: สาทร → ราชพฤกษ์,402,bus,14,"[0.518, 0.523, 0.66, 0.367, 0.459, 0.693, 2.45...",14.413588,"[[100.5307318, 13.7212134], [100.5328303, 13.7..."
4,2170840,203 สนามหลวง - ท่าอิฐ,203,bus,32,"[6.23, 1.017, 0.28, 1.123, 0.323, 0.503, 0.435...",79.780811,"[[100.4835788, 13.7709813], [100.5139682, 13.8..."
...,...,...,...,...,...,...,...,...
573,18791074,2-36 วัดคลองเสมียนตรา - ศูนย์ราชการฯ แจ้งวัฒนะ,2-36 Extra,bus,43,"[0.719, 1.057, 0.972, 1.421, 0.399, 0.175, 0.5...",42.469818,"[[100.2738601, 14.0236054], [100.2805052, 14.0..."
574,18791075,2-36 ศูนย์ราชการฯ แจ้งวัฒนะ - วัดคลองเสมียนตรา,2-36 Extra,bus,36,"[0.403, 0.554, 0.108, 7.561, 0.389, 0.214, 0.3...",42.449039,"[[100.5628995, 13.8820572], [100.5658821, 13.8..."
575,19149084,รถโดยสารประจำทางด่วนพิเศษ 402 : ราชพฤกษ์ → สาทร,402,bus,14,"[3.354, 0.6, 1.235, 1.196, 1.341, 0.955, 2.429...",14.329081,"[[100.4790492, 13.7157274], [100.5003151, 13.6..."
576,19553941,3-31 เมกาบางนา - สีลม,3-31 Extra,bus,28,"[0.667, 3.505, 1.352, 0.674, 2.898, 1.145, 0.4...",27.679866,"[[100.6753164, 13.6448315], [100.6798642, 13.6..."


SECTION 3: DATA PREPROCESSING


SECTION 3.1: HDBSCAN CLUSTERING ANALYSIS


In [ ]:
import hdbscan
print("="*80)
print("STEP 5: HDBSCAN CONGESTION AREA DETECTION")
print("="*80 + "\n")

# Get slow traffic (congestion indicator)
congested_traffic = traffic_df[traffic_df['speed'] < 15].copy()
print(len(congested_traffic))


STEP 5: HDBSCAN CONGESTION AREA DETECTION

1103905


In [ ]:
print(len(congested_traffic))

1103905


In [ ]:
if len(congested_traffic) > 200000:
    congested_traffic = congested_traffic.sample(n=200000,random_state=42)
coords = congested_traffic[['lat', 'lon']].values
coords_rad = np.radians(coords)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=50,
    min_samples=10,
    metric='haversine'
)

labels = clusterer.fit_predict(coords_rad)
congested_traffic['congestion_zone'] = labels


In [ ]:
congestion_zones = []
for zone_id in set(labels):
    if zone_id == -1:
        continue

    zone_data = congested_traffic[congested_traffic['congestion_zone'] == zone_id]

    congestion_zones.append({
        'zone_id': zone_id,
        'center_lat': zone_data['lat'].mean(),
        'center_lon': zone_data['lon'].mean(),
        'avg_speed': zone_data['speed'].mean(),
        'size': len(zone_data),
        'severity': 'Critical' if zone_data['speed'].mean() < 3.3 else 'High'
    })

congestion_zones_df = pd.DataFrame(congestion_zones)
print(f"Detected {len(congestion_zones_df)} congestion zones")
print(congestion_zones_df[['zone_id', 'center_lat', 'center_lon', 'avg_speed', 'severity']].to_string(index=False))

Detected 1361 congestion zones
 zone_id  center_lat  center_lon  avg_speed severity
       0   13.542769  100.876504   0.000000 Critical
       1   13.903586  100.866643   0.000000 Critical
       2   13.502421  100.741943   0.154639 Critical
       3   13.529985  100.831132   0.258065 Critical
       4   13.640945  100.855737   0.000000 Critical
       5   13.674266  100.788653   0.063492 Critical
       6   13.790898  100.869016   1.076923 Critical
       7   13.536198  100.786981   0.364865 Critical
       8   13.999784  100.457018   0.000000 Critical
       9   13.973404  100.782990   0.542373 Critical
      10   13.845507  100.805491   0.157143 Critical
      11   13.825418  100.866275   0.608696 Critical
      12   13.848777  100.860583   0.777778 Critical
      13   13.679406  100.570800   0.151163 Critical
      14   13.842827  100.840739   0.000000 Critical
      15   13.853004  100.839043   0.437500 Critical
      16   13.775092  100.837663   0.181818 Critical
      17   13.5

In [ ]:
congestion_zones_df.to_csv("/content/drive/MyDrive/DA_Project/cleaned/congestion_zones.csv")

In [ ]:
congestion_zones_df['avg_speed'].max()

6.707692307692308

SECTION 3.2: MAPPING CONGESTION AREA AND TRAFFIC POINTS

In [ ]:
from scipy.spatial import cKDTree

def haversine(lon1, lat1, lon2, lat2):
    """Calculate distance between two points (km)"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return c * 6371

def build_congestion_kdtree(congestion_zones_df):
    if congestion_zones_df.empty:
        return None, None

    coords = congestion_zones_df[['center_lat', 'center_lon']].values

    kdtree = cKDTree(coords)

    zone_data = congestion_zones_df[['zone_id', 'severity', 'avg_speed', 'size']].to_dict('records')

    return kdtree, zone_data

def is_near_congestion_kdtree(lat, lon, kdtree, zone_data, threshold_km=0.5):
    if kdtree is None:
        return 0, None, None  # no congestion zones

    threshold_degrees = threshold_km / 111.0  # ~ degrees per km

    distance, index = kdtree.query([lat, lon])

    # Convert distance (degrees) → km
    distance_km = distance * 111.0

    if distance <= threshold_degrees:
        zone_info = zone_data[index]
        return 1, zone_info, distance_km

    return 0, None, distance_km


def query_all_nearby_zones(lat, lon, kdtree, zone_data, threshold_km=0.5):

    if kdtree is None:
        return []

    threshold_degrees = threshold_km / 111.0

    # Query all points within radius
    indices = kdtree.query_ball_point([lat, lon], threshold_degrees)

    nearby_zones = [zone_data[i] for i in indices]

    return nearby_zones

In [ ]:
congestion_zones_df = pd.read_csv('/content/drive/MyDrive/DA_Project/cleaned/congestion_zones.csv')
print(f"✅ Loaded {len(congestion_zones_df)} congestion zones\n")

✅ Loaded 1361 congestion zones



In [ ]:
congestion_zones_df

,Unnamed: 0.1,Unnamed: 0,zone_id,center_lat,center_lon,avg_speed,size,severity
0,0,0,0,13.542769,100.876504,0.000000,96,Critical
1,1,1,1,13.903586,100.866643,0.000000,90,Critical
2,2,2,2,13.502421,100.741943,0.154639,194,Critical
3,3,3,3,13.529985,100.831132,0.258065,62,Critical
4,4,4,4,13.640945,100.855737,0.000000,82,Critical
...,...,...,...,...,...,...,...,...
1356,1356,1356,1356,13.745587,100.535130,1.039326,356,Critical
1357,1357,1357,1357,13.744332,100.541618,1.951220,82,Critical
1358,1358,1358,1358,13.744145,100.539818,1.570370,135,Critical
1359,1359,1359,1359,13.746647,100.540537,1.392857,56,Critical


SECTION 4: FEATURE ENGINEERING

In [ ]:
print("="*80)
print("STEP 3: FEATURE ENGINEERING WITH SPATIAL INDEX")
print("="*80 + "\n")

print(" Creating features...")

# Time features
traffic_df['timestamp'] = pd.to_datetime(traffic_df['timestamp'])
traffic_df['hour'] = traffic_df['timestamp'].dt.hour
traffic_df['day_of_week'] = traffic_df['timestamp'].dt.dayofweek
traffic_df['is_weekend'] = traffic_df['day_of_week'].isin([5, 6]).astype(int)
traffic_df['is_rush_hour'] = traffic_df['hour'].isin([7, 8, 17, 18, 19]).astype(int)

# Historical speeds
traffic_df['location_avg_speed'] = traffic_df.groupby('location_id')['speed'].transform('mean')
traffic_df['hour_avg_speed'] = traffic_df.groupby('hour')['speed'].transform('mean')

# Distance from center
bangkok_center_lat, bangkok_center_lon = 13.7563, 100.5018
traffic_df['distance_from_center'] = np.sqrt(
    (traffic_df['lat'] - bangkok_center_lat)**2 +
    (traffic_df['lon'] - bangkok_center_lon)**2
)

# NEW: Build KD-Tree spatial index for congestion zones
print("   Building KD-Tree spatial index for congestion zones...")

congestion_kdtree, congestion_zone_data = build_congestion_kdtree(congestion_zones_df)

if congestion_kdtree is not None:
    print(f"    KD-Tree built with {len(congestion_zone_data)} zones")

    # Use KD-Tree for ULTRA-FAST spatial queries
    print("   Applying spatial queries (blazing fast with cKDTree)...")

    congestion_results = traffic_df.apply(
    lambda row: is_near_congestion_kdtree(
        row['lat'], row['lon'],
        congestion_kdtree, congestion_zone_data,
        threshold_km=0.5
    ),
    axis=1
)

# Split tuple into separate columns
traffic_df['near_congestion'] = [r[0] for r in congestion_results]
traffic_df['congestion_severity'] = [r[1]['severity'] if r[1] else 'None' for r in congestion_results]
traffic_df['congestion_avg_speed'] = [r[1]['avg_speed'] if r[1] else 0 for r in congestion_results]
traffic_df['distance_to_congestion'] = [r[2] if r[2] is not None else np.nan for r in congestion_results]
traffic_df['distance_to_congestion'] = traffic_df['distance_to_congestion'].fillna(9999)


print(f"\n Features created")
print()

STEP 3: FEATURE ENGINEERING WITH SPATIAL INDEX

🔧 Creating features...
   Building KD-Tree spatial index for congestion zones...
    KD-Tree built with 1361 zones
   Applying spatial queries (blazing fast with cKDTree)...

 Features created



In [ ]:
traffic_df['congestion_severity_encode'] = np.where(traffic_df['congestion_severity'] == 'Critical', 2, 1)



In [ ]:
traffic_df.to_csv("/content/drive/MyDrive/DA_Project/processed/traffic.csv", index=False)

In [ ]:
traffic_df = pd.read_csv("/content/drive/MyDrive/DA_Project/processed/traffic.csv")

In [ ]:
traffic_df.columns

Index(['Unnamed: 0', 'VehicleID', 'gpsvalid', 'lat', 'lon', 'timestamp',
       'speed', 'heading', 'for_hire_light', 'engine_acc', 'hour',
       'day_of_week', 'is_weekend', 'is_rush_hour', 'lat_grid', 'lon_grid',
       'location_id', 'location_avg_speed', 'hour_avg_speed',
       'distance_from_center', 'near_congestion', 'congestion_severity',
       'congestion_severity_encode', 'congestion_avg_speed',
       'distance_to_congestion'],
      dtype='object')

In [ ]:
print("\n" + "="*80)
print("STEP 6: MACHINE LEARNING - PREDICTIVE ANALYTICS")
print("="*80 + "\n")

# Prepare features
feature_columns = ['lat', 'lon', 'heading', 'hour', 'day_of_week', 'is_weekend',
       'is_rush_hour', 'lat_grid', 'lon_grid', 'distance_from_center',
       'near_congestion', 'congestion_severity_encode', 'distance_to_congestion','congestion_avg_speed'
]
route_traffic = traffic_df
route_id = "000"
if len(route_traffic) > 200000:
    route_traffic = route_traffic.sample(n=200000, random_state=42)


X = route_traffic[feature_columns]
y = route_traffic['speed']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDataset Split:")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")

# Initialize and train the model
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nRandom Forest Regression:")
print(f"   MAE: {mae:.3f} km/h")
print(f"   R²: {r2:.3f}")


STEP 6: MACHINE LEARNING - PREDICTIVE ANALYTICS


Dataset Split:
   Training samples: 160,000
   Test samples: 40,000

Random Forest Regression:
   MAE: 11.991 km/h
   R²: 0.379


SECTION 6: MACHINE LEARNING MODELS (PREDICTIVE ANALYTICS)

In [ ]:
print("\n" + "="*80)
print("STEP 6: MACHINE LEARNING - PREDICTIVE ANALYTICS")
print("="*80 + "\n")

print("Training machine learning models to predict traffic speeds...")

# Prepare features
feature_columns = ['lat', 'lon', 'heading', 'hour', 'day_of_week', 'is_weekend',
       'is_rush_hour', 'lat_grid', 'lon_grid', 'distance_from_center',
       'near_congestion', 'congestion_severity_encode', 'distance_to_congestion','congestion_avg_speed'
]

X = traffic_df[feature_columns]
y = traffic_df['speed']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDataset Split:")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1,
        verbose=0
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100,
        max_depth=7,
        learning_rate=0.1,
        random_state=42,
        verbosity=0
    )
}

results = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"🤖 Training {name}...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[name] = {'model': model, 'mae': mae, 'r2': r2}
    print(f"   MAE: {mae:.3f} km/h | R²: {r2:.3f}")

# Select best model
best_model_name = min(results.keys(), key=lambda x: results[x]['mae'])
best_model = results[best_model_name]['model']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   MAE: {results[best_model_name]['mae']:.3f} km/h")
print(f"   R²: {results[best_model_name]['r2']:.3f}\n")



STEP 6: MACHINE LEARNING - PREDICTIVE ANALYTICS

Training machine learning models to predict traffic speeds...

Dataset Split:
   Training samples: 1,239,680
   Test samples: 309,920
🤖 Training Linear Regression...
   MAE: 15.459 km/h | R²: 0.105
🤖 Training Random Forest...


In [ ]:
# Feature importance (for tree-based models)
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\n📊 Feature Importances:")
    for idx, row in feature_importance.iterrows():
        print(f"   {row['feature']:<20} {row['importance']:.4f}")


📊 Feature Importances:
   distance_to_congestion 0.3060
   lat                  0.1558
   lon                  0.1417
   distance_from_center 0.1195
   heading              0.1076
   congestion_avg_speed 0.0868
   hour                 0.0455
   day_of_week          0.0143
   lon_grid             0.0087
   lat_grid             0.0077
   is_rush_hour         0.0038
   is_weekend           0.0017
   congestion_severity_encode 0.0008
   near_congestion      0.0001
